In [ ]:
import h5py 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from Bio import SeqIO

sns.set(style='white')

In [ ]:
'''
make a df: distance vs attribution codon wise (with and without absolute values)
'''
record_tr = []
sequence = []

for record in SeqIO.parse("/net/lts2gdk0/mnt/scratch/lts2/nallapar/rb-prof/data/Jan_2024/Lina/reference/ensembl.cds.fa", "fasta"):
    record_tr.append(str(record.description.split(' ')[0]))
    sequence.append(str(record.seq))

# make df
df = pd.DataFrame({'Transcript': record_tr, 'Sequence': sequence})

print("Made Transcript-Sequence DF")

# convert sequence into codon triplets list
df['Codon'] = df['Sequence'].apply(lambda x: [x[i:i+3] for i in range(0, len(x), 3)])

df_test = pd.read_csv('predictions/exp_dat_testLiver.csv')
test_tr = list(df_test['transcript'])

int_ds = h5py.File('/net/lts2gdk0/mnt/scratch/lts2/nallapar/rb-prof/data/Jan_2024/Lina/full_Test_LIXG_RiboGL.h5', 'r')
len_samples = len(int_ds['node_attr'])
int_tr = list(int_ds['transcript'])

codon = []
distance_ASite = []
attribution_vals = []
attribution_abs = []

for i in tqdm(range(len_samples)):
    node_attr_sample = int_ds['node_attr'][i]
    # num codons is square root of length of node_attr_sample
    num_codons = int(len(node_attr_sample)**0.5)
    # convert to 2D array of shape (num_codons, num_codons)
    node_attr_sample = node_attr_sample.reshape(num_codons, num_codons)
    # get codon sequence for this transcript
    codon_seq = df[df['Transcript'] == str(int_tr[i].decode('utf-8'))]['Codon'].values[0]

    for j in range(num_codons):
        sample_attr = node_attr_sample[j] / np.max(np.abs(node_attr_sample[j]))
        sample_attr_mod = np.abs(node_attr_sample[j]) / np.max(np.abs(node_attr_sample[j]))
        for k in range(num_codons):
            attribution_vals.append(sample_attr[k])
            attribution_abs.append(sample_attr_mod[k])
            distance_ASite.append(k-j)
            codon.append(codon_seq[k])

data = {'Distance from A Site': distance_ASite, 'Attribution Normal': attribution_vals, 'Attribution Mod': attribution_abs, 'Codon': codon}
df_distCodonAttr = pd.DataFrame(data)

df_distCodonAttr.to_csv('final_plots/distCodonAttr.csv')

print("Saved DF with Distance from A Site vs Attribution (Normal and Mod) for each Codon")

In [ ]:
'''
Global Attribution Plot
'''
# make line plot with seaborn
# set size of plot
plt.figure(figsize=(10, 6))
sns.lineplot(x=df_distCodonAttr['Distance from A Site'], y=df_distCodonAttr['Attribution Mod'], ci='sd', color='#eb4d4b')
# crop plot to only show relevant data from -500 to 500
plt.xlim(-500, 500)
# add vertical bar on x=0
plt.axvline(x=0, color='black', linestyle='--')
# x and y labels
plt.xlabel('Offset from A site (codons)', fontsize=15)
plt.ylabel('Attribution', fontsize=15)

plt.title('Contribution of positions in the global neighborhood of the A site', fontsize=20)
plt.savefig('final_plots/distance_vs_attribution_500Global.svg')
plt.savefig('final_plots/distance_vs_attribution_500Global.png')

plt.show()

In [ ]:
'''
local attribution plot from -7 to +7
'''

# box plot with seaborn from -7 to 7 
df_local = df_distCodonAttr[(df_distCodonAttr['Distance from A Site'] >= -7) & (df_distCodonAttr['Distance from A Site'] <= 7)]
plt.figure(figsize=(10, 6))
# barplot for the values from -7 to 7
sns.barplot(x=df_local['Distance from A Site'], y=df_local['Attribution Mod'], color='#eb4d4b')
plt.ylim(0, 0.55)

plt.xlabel('Offset from A site (codons)', fontsize=15)
plt.ylabel('Attribution', fontsize=15)
# change x ticks to be from -7 to 7
plt.xticks(range(0, 15), ['-7', '-6', '-5', '-4', '-3', 'E', 'P', 'A', '1', '2', '3', '4', '5', '6', '7'])
# change y ticks to remove the last value, make that blank
plt.yticks(np.arange(0, 0.6, 0.1), ['0', '0.1', '0.2', '0.3', '0.4', ''])
# plot title
plt.title('Contribution of positions in the local neighborhood of the A site', fontsize=20)
plt.savefig('final_plots/distance_vs_attribution_barALocal.svg')
plt.savefig('final_plots/distance_vs_attribution_barALocal.png')

plt.show()

In [ ]:
'''
make codon wise attribution plot
'''

# convert codon column to list
# df['Codon'] = df['Codon'].apply(lambda x: x[1:-1].split(','))

# only take positions -5 to 5
df_localCodon = df_distCodonAttr[(df_distCodonAttr['Distance from A Site'] >= -5) & (df_distCodonAttr['Distance from A Site'] <= 5)]

# remove codon with len>3
df_localCodon = df_localCodon[df_localCodon['Codon'].apply(lambda x: len(x) == 3)]

# codon to int dictionary
codon_to_id = {
    'GCG': 0,
    'GCA': 1,
    'GCC': 2,
    'GCT': 3,
    'TGT': 4,
    'TGC': 5,
    'GAC': 6,
    'GAT': 7,
    'GAG': 8,
    'GAA': 9,
    'TTT': 10,
    'TTC': 11,
    'GGC': 12,
    'GGT': 13,
    'GGA': 14,
    'GGG': 15,
    'CAT': 16,
    'CAC': 17,
    'ATC': 18,
    'ATA': 19,
    'ATT': 20,
    'AAG': 21,
    'AAA': 22,
    'TTA': 23,
    'TTG': 24,
    'CTT': 25,
    'CTA': 26,
    'CTC': 27,
    'CTG': 28,
    'ATG': 29,
    'AAC': 30,
    'AAT': 31,
    'CCT': 32,
    'CCC': 33,
    'CCA': 34,
    'CCG': 35,
    'CAG': 36,
    'CAA': 37,
    'CGG': 38,
    'AGA': 39,
    'CGA': 40,
    'AGG': 41,
    'CGC': 42,
    'CGT': 43,
    'TCT': 44,
    'TCC': 45,
    'TCA': 46,
    'TCG': 47,
    'AGT': 48,
    'AGC': 49,
    'ACT': 50,
    'ACG': 51,
    'ACC': 52,
    'ACA': 53,
    'GTA': 54,
    'GTC': 55,
    'GTT': 56,
    'GTG': 57,
    'TGG': 58,
    'TAC': 59,
    'TAT': 60,
    'TAA': 61,
    'TGA': 62,
    'TAG': 63
}

id_to_codon = {v:k for k,v in codon_to_id.items()}

# convert codon to int
codon_list = list(df_localCodon['Codon'])
codonintlist = []
for codon in codon_list:
    codonintlist.append(codon_to_id[codon])
df_localCodon['CodonInt'] = codonintlist
# drop codon column
df_localCodon = df_localCodon.drop('Codon', axis=1)
df_localCodon = df_localCodon.drop('Unnamed: 0', axis=1)

# make a heatmap with seaborn, with the mean attribution for each codon at each position    
df_localCodon = df_localCodon.groupby(['Distance from A Site', 'CodonInt']).mean().reset_index()
# make a table 15 x 64 with the mean attribution for each codon at each position
df_table = df_localCodon.pivot(index='Distance from A Site', columns='CodonInt', values='Attribution')
plt.figure(figsize=(40, 10))
# make a heatmap with seaborn using the table
# cmap negative with red, positive with blue
# print full df_table
# save df table
df_table.to_csv('df_table.csv')
# impute nans to 0
df_table = df_table.fillna(0)
# invert the y axis
df_table = df_table.iloc[::-1]

# remove last three rows
df_table = df_table.iloc[:,:-3]

ax = sns.heatmap(df_table, cmap='coolwarm', center=0, cbar_kws={'label': ''}, square=True, annot_kws={"size": 40})
ax.figure.axes[-1].yaxis.label.set_size(40)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
cbar.ax.tick_params(labelsize=25)
plt.xlabel('', fontsize=40)
plt.ylabel('Offset from A site (codons)', fontsize=40)
plt.title('Contribution of codons in different positions around the A site', fontsize=40)

# flip y axis
# change y axis 
plt.yticks(range(0, 11), ['5', '4', '3', '2', '1', 'A', 'P', 'E', '-3', '-4', '-5'], fontsize=30, va='top', rotation=0)

codon_to_aa = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
        'NNG':'R', 'NNC':'T', 'NGT':'S', 'NGA':'R',
        'NNT':'Y', 'NGC':'S'
    }

# group codons by amino acid and order the x ticks
aa_list = []
codon_list = []
for i in range(61):
    aa_list.append(codon_to_aa[id_to_codon[i]])
    codon_list.append(id_to_codon[i])

plt.xticks(range(0, 61), codon_list, rotation=90, fontsize=30, ha='left')

plt.savefig('final_plots/distance_vs_attribution_heatmapCodon.png')
plt.show()